<h2>Congreso COMIA-Taller</h2>

### Análisis exploratorio 

In [25]:
#Librerias
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import random

### Construyendo la base de datos municipal

In [26]:
# Complejidad: crecimiento a largo plazo
# Educación: ANUIES+ENOE+ población alfabeta+Niveles educativos

#Tecnologia: inclusión tecnológica+Investigación

#Infraestructura: infraestructura publica para la vivienda +índice de marginación+ Aeropuertos+puertos+pasajeros+vuelos
#Acumulación de capital: 

#Lista de municipios para contruir la base de datos
municipios=pd.read_csv("Municipios.csv")
municipios["Code_INEGI"]=municipios["Code_INEGI"].astype(str).str.zfill(5)
municipios["Code_ent"]=municipios["Code_INEGI"].str[:2]
municipios

,Code_INEGI,Municipio,Code_ent
0,01001,Aguascalientes,01
1,01002,Asientos,01
2,01003,Calvillo,01
3,01004,Cosío,01
4,01010,El Llano,01
...,...,...,...
2464,32052,Villa García,32
2465,32053,Villa González Ortega,32
2466,32054,Villa Hidalgo,32
2467,32055,Villanueva,32


In [28]:
#Agregando la ANUIES IMPUTADA
ANUIES=pd.read_csv("COMIA-datos - anuies_imputado.csv")
ANUIES=ANUIES.rename(columns = {'CVEGEO':'Code_INEGI'})
ANUIES["Code_INEGI"]=ANUIES["Code_INEGI"].astype(str).str.zfill(5)
municipios=municipios.merge(ANUIES,on='Code_INEGI', how='left')

In [29]:
#ENOE
ENOE=pd.read_csv("ENOE_Municipality-Industry Group.csv")
ENOE["Code_INEGI"]=ENOE["Code_INEGI"].astype(str).str.zfill(5)
grupos=ENOE["Industry Group"].unique()

for igroup in grupos:
    df_grupo=ENOE[ENOE["Industry Group"]==igroup]
    df_grupo=df_grupo.rename(columns = {'Workforce':'ENOE_'+str(igroup)})
    df_grupo=df_grupo.drop(columns=['Industry Group ID','Municipality',"Industry Group"])
    municipios=municipios.merge(df_grupo,on='Code_INEGI', how='left')

In [30]:
#Población alfabeta
Alfabeta=pd.read_csv("COMIA-datos - Censo_poblacion_alfabeta.csv")
Alfabeta["Codigo_INEGI"]=Alfabeta["Codigo_INEGI"].astype(str).str.zfill(5)
#grupos=ENOE["Industry Group"].unique()
Alfabeta=Alfabeta[Alfabeta["Alfabeta"]=="Yes"]
Alfabeta=Alfabeta.rename(columns = {'Codigo_INEGI':'Code_INEGI','poblacion':'poblacion_alfabeta'})
Alfabeta=Alfabeta.drop(columns=['ID_Alfabeta','Muncipio',"Alfabeta"])
municipios=municipios.merge(Alfabeta,on='Code_INEGI', how='left')

In [31]:
#grado de estudios
grado_estudios=pd.read_csv("COMIA-datos - Censo_grado_de_estudios.csv")
grado_estudios["Codigo_INEGI"]=grado_estudios["Codigo_INEGI"].astype(str).str.zfill(5)
grupos=grado_estudios["Grado_Academico"].unique()

for igroup in grupos:
    df_grado=grado_estudios[grado_estudios["Grado_Academico"]==igroup]                          
    df_grado=df_grado.rename(columns = {'Poblacion':'Censo_'+str(igroup),'Codigo_INEGI':'Code_INEGI'})                         
    df_grado=df_grado.drop(columns=['ID_Grado_academico','Muncipio','Grado_Academico'])
    municipios=municipios.merge(df_grado,on='Code_INEGI', how='left')

In [32]:
#IMSS

#Agricultura, ganadería, silvicultura, pesca y caza-11
IMSS=pd.read_csv("IMSS-Municipality-Economic Sector 1 digit.csv")
IMSS["Municipality ID"]=IMSS["Municipality ID"].astype(str).str.zfill(5)
sectores_imss=IMSS["Economic Sector 1 digit"].unique()
for sector in sectores_imss:
    df_sec=IMSS[IMSS["Economic Sector 1 digit"]==sector]
    df_sec=df_sec.drop(columns=["Municipality","Economic Sector 1 digit","Economic Sector 1 digit ID"])
    df_sec=df_sec.rename(columns = {'Insured People':'IMSS_'+str(sector),'Municipality ID':'Code_INEGI'})
    municipios=municipios.merge(df_sec,on='Code_INEGI', how='left')

In [33]:
#ventas por internet
ventas=pd.read_csv("COMIA-datos - unidades_economicas_con_ventas_por_internet.csv")
ventas["entidad"]=ventas["entidad"].astype(str).str.zfill(2)
subsectores=ventas["subsector"].unique()

for subsec in subsectores:
    subsector=ventas[ventas["subsector"]==subsec]
    subsector=subsector.rename(columns = {'UE_sin_ventas':'economico_UE_sin_ventas_'+str(subsec),
                                        'UE_con_ventas':'economico_UE_con_ventas_'+str(subsec),
                                        'entidad':'Code_ent'}) 
    subsector=subsector.drop(columns=['subsector'])
    municipios=municipios.merge(subsector,on='Code_ent', how='left')
    

In [34]:
#Censo viviendas infraestructura
viviendas=pd.read_csv("COMIA-datos - Censo_viviendas_infraestructura.csv")
viviendas["Code_INEGI"]=viviendas["Code_INEGI"].str.replace(' ','')
viviendas=viviendas.drop(columns=['Municipio','Total'])
municipios=municipios.merge(viviendas,on='Code_INEGI', how='left')

In [35]:
#marginacion
marginacion=pd.read_csv("COMIA-datos - CONAPO_Indice_de_marginacion.csv")
marginacion=marginacion[["CVE_MUN","IM_2020","GM_2020"]]
marginacion["CVE_MUN"]=marginacion["CVE_MUN"].astype(str).str.zfill(5)
marginacion=marginacion.rename(columns={'CVE_MUN':"Code_INEGI","IM_2020":"CONAPO_IM_2020","GM_2020":"CONAPO_GM_2020"})
municipios=municipios.merge(marginacion,on='Code_INEGI', how='left')


In [36]:
#aeropuertos
aeropuertos=pd.read_csv("COMIA-datos - aeropuertos_imputado_municipios.csv")
aeropuertos["cve_ent"]=aeropuertos["cve_ent"].astype(str).str.zfill(2)
aeropuertos["cve_mun"]=aeropuertos["cve_mun"].astype(str).str.zfill(3)
aeropuertos["Code_INEGI"]=aeropuertos["cve_ent"]+aeropuertos["cve_mun"]
aeropuertos=aeropuertos[["Code_INEGI","conteo_aeropuertos"]]
municipios=municipios.merge(aeropuertos,on='Code_INEGI', how='left')

In [37]:
#pasajeros
pasajeros=pd.read_csv("COMIA-datos - pasajeros_imputado_municipios.csv")
pasajeros["cve_ent"]=pasajeros["cve_ent"].astype(str).str.zfill(2)
pasajeros["cve_mun"]=pasajeros["cve_mun"].astype(str).str.zfill(3)
pasajeros["Code_INEGI"]=pasajeros["cve_ent"]+pasajeros["cve_mun"]
pasajeros=pasajeros[["Code_INEGI","pasajeros_2019"]]
municipios=municipios.merge(pasajeros,on='Code_INEGI', how='left')

In [38]:
#vuelos
vuelos=pd.read_csv("COMIA-datos - vuelos_imputado_municipíos.csv")
vuelos["cve_ent"]=vuelos["cve_ent"].astype(str).str.zfill(2)
vuelos["cve_mun"]=vuelos["cve_mun"].astype(str).str.zfill(3)
vuelos["Code_INEGI"]=vuelos["cve_ent"]+vuelos["cve_mun"]
vuelos=vuelos[["Code_INEGI","vuelos_2019"]]
municipios=municipios.merge(vuelos,on='Code_INEGI', how='left')

In [39]:
#Ahora agregamos los ECI

ECI=pd.read_csv("COMIA-datos - eci_2015_2020_denue.csv")
ECI=ECI.rename(columns={'cvegeo':'Code_INEGI'})
ECI=ECI.drop(columns=['nombre'])
ECI["Code_INEGI"]=ECI["Code_INEGI"].astype(str).str.zfill(5)
municipios=municipios.merge(ECI,on='Code_INEGI', how='left')

In [40]:
#guardando la base de datos
municipios.to_csv("base_de_datos_municipio.csv",index=False)

In [41]:
#venganos la imputación
#matriz de distancias
centroides=pd.read_csv("cenotrides.csv")
centroides.index=centroides["CVEGEO"]

#distancia euclidiana
from scipy.spatial import distance
matriz_distance=pd.DataFrame(distance.cdist(centroides[["x","y"]],centroides[["x","y"]]))

matriz_distance.index=centroides["CVEGEO"]
matriz_distance= matriz_distance.set_axis(centroides["CVEGEO"],axis=1)

In [43]:
matriz_distance.to_csv("distancia.csv")